In [1]:
import pandas as pd
import geopandas as gpd

In [20]:
txt = "Data/Traverse.txt"
df = pd.read_csv(txt, sep = "\t")

In [21]:
df.nunique()
df.drop(columns = ['desc', 'kml_folder', 'type'], inplace = True)

df.rename(columns={'name': 'description'}, inplace=True)
df['description'].ffill(inplace=True)


In [39]:
gdf = gpd.GeoDataFrame(data = df,
                        crs = "EPSG:4326",
                        geometry = gpd.points_from_xy(df['longitude'], df['latitude'])) # type: ignore

gdf = gdf.to_crs('EPSG:32737')                        

gdf.drop(columns = ['latitude', 'longitude'], inplace = True)

In [40]:
gdf.head(10)

,altitude (m),description,geometry
0,1522.2,Traverse,POINT (278749.445 9878773.131)
1,1524.6,Traverse,POINT (278884.378 9878764.330)
2,1525.0,Traverse,POINT (278931.200 9878773.030)
3,1523.6,Traverse,POINT (278923.524 9878539.287)
4,1521.0,Traverse,POINT (278835.170 9878515.318)
5,1520.7,Traverse,POINT (278807.397 9878451.506)
6,1521.3,Traverse,POINT (278864.796 9878417.275)
7,1521.5,Traverse,POINT (278883.217 9878347.821)
8,1522.4,Traverse,POINT (278966.088 9878354.763)
9,1522.5,Traverse,POINT (278976.174 9878413.624)


In [41]:
e = 1618010.606367111
f = -1741978.211608887

a = 0.9916765364496314
d = 1.175648448821448

b = -0.1635616585599564
c = 0.02551828620198648

# Transform to corresponding coordinates
gdf2 = gpd.GeoDataFrame((gdf.affine_transform([a, b, c, d, e, f])))
# gdf2.set_crs("EPSG:32737")

In [42]:
gdf2.head(30)

,0
0,POINT (278651.372 9879099.305)
1,POINT (278786.622 9879092.401)
2,POINT (278831.632 9879103.823)
3,POINT (278862.250 9878828.828)
4,POINT (278778.552 9878798.395)
5,POINT (278761.448 9878722.665)
6,POINT (278823.968 9878683.886)
7,POINT (278853.596 9878602.703)
8,POINT (278934.642 9878612.979)
9,POINT (278935.016 9878682.436)
